In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error

model1 = pd.read_csv('../../data/fivePCModel.csv',index_col='Date',parse_dates=True)
model2 = pd.read_csv('../../data/PCModel2.csv',index_col='Date',parse_dates=True)

In [52]:
model1.head()

,Zipcode,City,State,Metro,County,SizeRank,Rent,Year,State-County,PC1,PC2,PC3,PC4,PC5
Date,,,,,,,,,,,,,,
2015-01-01,10025,New York,NY,New York-Newark-Jersey City,New York County,1,3566.0,2015,NY-New York County,0.375709,-0.477269,4.222821,-1.427123,-1.455476
2015-01-01,10023,New York,NY,New York-Newark-Jersey City,New York County,3,3413.0,2015,NY-New York County,3.035421,-1.039616,3.481363,-1.740920,-0.635692
2015-01-01,10002,New York,NY,New York-Newark-Jersey City,New York County,7,3508.0,2015,NY-New York County,-1.782419,0.958593,2.972139,-1.500729,-0.784467
2015-01-01,11226,New York,NY,New York-Newark-Jersey City,Kings County,11,1876.0,2015,NY-Kings County,-2.764733,-1.508810,2.286786,-0.982683,-1.140958
2015-01-01,10467,New York,NY,New York-Newark-Jersey City,Bronx County,12,1442.0,2015,NY-Bronx County,-4.388430,-0.423407,3.130730,-0.076299,-0.329126


In [48]:
LE = LabelEncoder() #sklearn.preprocessing

def preProc(zillow_df, ytype='log'):
    features = zillow_df.drop(['Rent','State-County','State'],axis=1)
    cat_cols = features.select_dtypes(include = object).copy()
    num_cols = features.select_dtypes(exclude = object).copy()
    for col in cat_cols.columns:
        cat_cols[col] = LE.fit_transform(cat_cols[col])
    
    X = pd.concat([cat_cols,num_cols],axis=1)
    
    if ytype == 'log':
        y = np.log(zillow_df['Rent'])
    elif ytype == 'normal':
        y = zillow_df['Rent']
    return X,y

In [49]:
X,y = preProc(model1,'log')

In [51]:
Xtrain = X.loc[X.index <'2019-01-01']
train_index = Xtrain.shape[0]
Xtest = X[train_index:]
ytrain = y[:train_index]
ytest = y[train_index:]

In [70]:
def randForest(model,Xtrain,Xtest,ytrain,ytest):
    model.fit(Xtrain,ytrain)
    print(f'training R2: {model.score(Xtrain,ytrain)}')
    print(f'test R2: {model.score(Xtest,ytest)}')
    
    ypred = rfc.predict(Xtest)
    RMSE = mean_squared_error(ytest,ypred, squared = False)
    print(f'RMSE: {RMSE}')
    
    feature_imps = pd.DataFrame({'Columns':Xtrain.columns,'Feature_importances':model.feature_importances_})
    return feature_imps.sort_values('Feature_importances',ascending=False)

In [71]:
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor()

randForest(rfc,Xtrain,Xtest,ytrain,ytest)

training R2: 0.9988398439019632
test R2: 0.9792883532558285
RMSE: 0.04079037470119717


,Columns,Feature_importances
1,Metro,0.393796
6,PC1,0.294761
3,Zipcode,0.196494
2,County,0.035391
9,PC4,0.020974
8,PC3,0.015757
5,Year,0.012606
7,PC2,0.012321
4,SizeRank,0.009550
0,City,0.004262


In [58]:
mini_train = Xtrain[['PC1','PC2','PC3','PC4','PC5']]
mini_test = Xtest[['PC1','PC2','PC3','PC4','PC5']]

In [72]:
randForest(rfc,mini_train,mini_test,ytrain,ytest)

training R2: 0.9977768250651446
test R2: 0.9383261818255458
RMSE: 0.07038835208147487


,Columns,Feature_importances
0,PC1,0.361624
1,PC2,0.259888
2,PC3,0.186450
3,PC4,0.127496
4,PC5,0.064542


In [53]:
model2.head()

,Zipcode,City,State,Metro,County,SizeRank,Rent,Year,State-County,Vol_moderate_income,...,PC2,PC3,secondPC1,secondPC2,PersonalIncome,total_pop,households,gini_index,housing_availability,move_within_city
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,10025,New York,NY,New York-Newark-Jersey City,New York County,1,3566.0,2015,NY-New York County,3.825176e+09,...,0.355586,0.375299,-0.785401,1.487250,-0.483885,2.052326,3.096986,2.090545,0.165373,0.557787
2015-01-01,10023,New York,NY,New York-Newark-Jersey City,New York County,3,3413.0,2015,NY-New York County,3.825176e+09,...,0.614175,0.221742,-0.728499,1.524861,-0.483885,0.708930,2.065297,1.450117,0.768800,0.220614
2015-01-01,10002,New York,NY,New York-Newark-Jersey City,New York County,7,3508.0,2015,NY-New York County,3.825176e+09,...,1.643955,1.894220,-1.243924,1.217707,-0.483885,1.264424,2.046087,1.861821,-0.524889,-0.408679
2015-01-01,11226,New York,NY,New York-Newark-Jersey City,Kings County,11,1876.0,2015,NY-Kings County,3.825176e+09,...,-0.073192,-0.264804,-2.165280,0.661003,-0.483885,2.382845,2.446985,-0.261799,-0.271246,-0.598288
2015-01-01,10467,New York,NY,New York-Newark-Jersey City,Bronx County,12,1442.0,2015,NY-Bronx County,3.825176e+09,...,0.575126,-0.253722,-2.373168,0.750569,-0.483885,2.535674,2.446860,0.053137,-0.650615,0.799210


In [67]:
x2,y2 = preProc(model2,'log')
X2train = x2.loc[x2.index <'2019-01-01']
train_index = X2train.shape[0]
X2test = x2[train_index:]
y2train = y2[:train_index]
y2test = y2[train_index:]

In [68]:
randForest(rfc,X2train,X2test,y2train,ytest)

training R2: 0.9994701701004696
test R2: 0.9813419004994669


,Columns,Feature_importances
1,Metro,0.392856
8,PC1,0.195652
11,secondPC1,0.131733
3,Zipcode,0.083705
17,housing_availability,0.053208
16,gini_index,0.043275
10,PC3,0.019375
9,PC2,0.016912
2,County,0.009987
13,PersonalIncome,0.008707
